In [1]:
#Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy.matlib as matlib
import glob, os
from numpy import linalg as LA
from scipy import sparse
from scipy.linalg import cholesky

#Augmented dickey fuller test library
from statsmodels.tsa.stattools import adfuller

In [129]:
class CurrencyTimeSeries:
    #oDates: Original dates, oPrices: Original Prices, transformed: transformed prices
    def __init__(self, tsLabel, oDates, oPrices, transformed=None, transfType=None):
        self.label = tsLabel
        #values used to rebuild the original time series
        self.firstValue = oPrices.iloc[0]
        self.dates = oDates
        self.prices = oPrices
        self.tPrices = transformed[~pd.isnull(transformed)]
        #self.tPrices = np.nan_to_num(transformed)
        self.transfType = transfType

In [133]:
def rebuildFirstDiffT(ts, firstValue):
    #print('Re-construyendo...')
    first_element_fts = firstValue
    #Rebuilding:
    #fts_diff_cumsum = ts.tPrices[~pd.isnull(ts.tPrices)].cumsum()
    fts_diff_cumsum = ts.cumsum()
    #print('fd-sum', fts_diff_cumsum)
    rebuilt = fts_diff_cumsum + first_element_fts
    #print('Re-built: ',rebuilt)
    return rebuilt

In [139]:
def levelT(y, x, n):
    y[:] = x.to_numpy()

#First difference: x(t)-x(t-1)
def firstDiffT(y, x, n):
    y[1:] = x.diff().dropna().to_numpy()

#Second difference: (x(t)-x(t-1))-(x(t-1)-x(t-2))
def secondDiffT(y, x, n):
    y[2:] = x[2:n].to_numpy()-2*(x[1:n-1].to_numpy())+x[0:n-2].to_numpy()

#Función de transformación de los datos para volverlos estacionarios
#Data: data to be transformed to be stationary
#tcode: transformation code
def transformData(Data, tcode):
    #Numero de observaciones (filas)
    [n, ]= Data.shape
        
    
    # Allocate output variable
    y = np.array([None] * n);
    
    
    
    # Se realiza la transformación #

    switcher = {
           1: levelT,
           2: firstDiffT,
           3: secondDiffT        
    }
       
    func = switcher.get(tcode, "nothing")
       
    func(y, Data, n)
    
    return y
    
#Function to standardize the data
def standardize(y):
    
    T = np.shape(y)[0]
    mean = np.mean(y, axis = 0)
    std = np.std(y, axis = 0)
    
    #my = matlib.repmat(mean, T, 1)
    #sy = matlib.repmat(std, T, 1)
       
    x = (y - mean)/std
    
    return x, mean, std

def reverseStandardization(x, mean, std):   
    
    y = x * (std) + mean
    
    return y

In [148]:
#Calling functions
h = 12
#Preparamos la data

df = pd.read_excel (r'../Data/CurrencyDataFrame.xlsx')

#df.iloc[row_start:row_end , col_start, col_end]
#Contiene las series de tiempo para varias monedas
Data = df.iloc[1:-h , 1:]
real_values = df.iloc[-h:,1:]
print(Data)
print(real_values)
#Codigo de transformación para cada ts
tcode = df.iloc[:1, 1:]
#Fechas de las observaciones
dates = df.iloc[:0,:1]
#Labels
labels = df.iloc[:0, 1:]

#Tamaño de la matriz de Data, m: filas, n: columnas
[m,n] = Data.shape 

# Transform variables, tiene el mismo tamaño que el de los datos. Es una matriz de ceros
Y = np.zeros((m,n)); 


#Transformando la data
#ciclo para recorrer todas las columnas
#transformData(Data.iloc[:,6],tcode.iloc[0,6])
for i in range (0, n):
    Y[:,i] = transformData(Data.iloc[:,i],tcode.iloc[0,i])
    ts = CurrencyTimeSeries(labels.iloc[:,i], None, Data.iloc[:,i], Y[:,i], tcode.iloc[0,i])
    
    ts_stand, mean, std = standardize(ts.tPrices)
    print('Standard',ts_stand)
    reverse_ts_stand = reverseStandardization(ts_stand, mean, std)
    #print('adding zero at the beggining')
    #print(np.append([0], reverse_ts_stand))
    print('Reverse Standard',reverse_ts_stand)
    
    original = rebuildFirstDiffT(reverse_ts_stand, ts.firstValue)
    #print(ts.label, ts.tPrices)
    print('Original', original)
    #if tcode.iloc[0,i] > 1:
    #    rebuildFirstDiffT(ts)
    
#Removing first observation as some values for some transformations are null (None)
Y = Y[1:]



      AUDCAD   AUDCHF     AUDJPY   AUDNZD  BGNRON   CADCHF     CADJPY  CHFBGN  \
1    0.98708  0.73898  86.539001  1.07680  2.3804  0.74856  87.653999  1.6901   
2    0.98214  0.73600  85.344002  1.08138  2.3795  0.75008  86.875000  1.7057   
3    0.98041  0.72815  84.505997  1.07768  2.3841  0.74264  86.189003  1.6997   
4    0.98430  0.73472  85.077003  1.07912  2.3794  0.74634  86.425003  1.6942   
5    0.98858  0.73768  85.413002  1.08110  2.3831  0.74602  86.389999  1.6939   
..       ...      ...        ...      ...     ...      ...        ...     ...   
111  0.97432  0.73582  82.147003  1.08742  2.3820  0.75460  84.289001  1.6819   
112  0.97240  0.73290  82.449997  1.08950  2.3810  0.75340  84.750000  1.6723   
113  0.97488  0.74264  83.435997  1.09270  2.3788  0.76161  85.568001  1.6708   
114  0.97606  0.74340  83.387001  1.09653  2.3805  0.76167  85.420998  1.6757   
115  0.97400  0.73901  83.313004  1.09267  2.3783  0.75867  85.529999  1.6771   

         CHFJPY  CHFRON  ..

 -1.07651999 -0.22396063  1.61694513  0.01183239 -1.04632203 -0.38430592]
Reverse Standard [ 5.32000e-02  9.67010e-02  1.96000e-02 -1.57010e-02  1.81000e-02
 -2.11010e-02 -2.80990e-02  8.70000e-03 -5.07000e-02  2.66000e-02
  2.61000e-02  8.29900e-03 -9.90000e-05  4.10100e-03  5.30990e-02
  1.65990e-02  3.20100e-03 -8.18010e-02  5.82010e-02 -2.25010e-02
  2.95010e-02 -5.50100e-03  6.07020e-02 -2.49010e-02  4.19010e-02
 -6.28010e-02 -2.27000e-02  9.89900e-03 -3.99900e-03  4.60000e-03
 -2.45000e-02  2.50000e-02  2.09010e-02  2.26000e-02  2.06980e-02
 -3.41980e-02 -7.27010e-02  2.73990e-02 -3.08000e-02 -5.48000e-02
  2.30000e-03  2.29000e-02  4.78000e-02 -3.87990e-02 -4.41000e-02
  2.37010e-02 -9.00100e-03 -1.52000e-02 -4.47000e-02  2.95010e-02
  1.59800e-03  3.38000e-02  4.73010e-02  5.70010e-02  1.82000e-02
  9.30000e-03  2.83980e-02 -2.52000e-02  1.27301e-01  8.87010e-02
 -8.21020e-02 -9.20980e-02 -3.30010e-02  5.39000e-02  4.82000e-02
 -2.30000e-03 -9.28000e-02  1.19000e-02  1.98000e-0